In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# from tensorflow import Tensor

%matplotlib inline

#  Import Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print(type(mnist))
mnist.train.images.shape

<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>


(55000, 784)

#  Clean

In [4]:
#  validation will be the labaled instances.... train will be the larger number of unlabaled instances
#  test will be the test instances in both cases

In [5]:
print(mnist.validation.images.shape)
print(mnist.train.images.shape)
print(mnist.test.images.shape)

(5000, 784)
(55000, 784)
(10000, 784)


In [6]:
def keep_n_labels(passed_data, n):
    data = np.copy(passed_data)
    all_indexes = np.asarray(list(range(len(data))))
    np.random.shuffle(all_indexes)
    if n >= 1:
        selected_indexes_to_set_to_0 = np.asarray(list(all_indexes[:-n]))
    else:
        selected_indexes_to_set_to_0 = np.asarray(list(all_indexes[:]))
#     normal_indexes = np.asarray(list(all_indexes[-n:]))
    data[selected_indexes_to_set_to_0] = np.zeros(data.shape[1])
    return data


In [7]:
mnist_train_y_0_labeled = keep_n_labels(mnist.train.labels, 0)
mnist_train_y_10_labeled = keep_n_labels(mnist.train.labels, 10)
mnist_train_y_50_labeled = keep_n_labels(mnist.train.labels, 50)
mnist_train_y_100_labeled = keep_n_labels(mnist.train.labels, 100)
mnist_train_y_200_labeled = keep_n_labels(mnist.train.labels, 200)
mnist_train_y_500_labeled = keep_n_labels(mnist.train.labels, 500)
mnist_train_y_1000_labeled = keep_n_labels(mnist.train.labels, 1000)
mnist_train_y_5000_labeled = keep_n_labels(mnist.train.labels, 5000)
mnist_train_y_10000_labeled = keep_n_labels(mnist.train.labels, 10000)
mnist_train_y_all_labeled = np.copy(mnist.train.labels)



print(np.sum(mnist_train_y_0_labeled))
print(np.sum(mnist_train_y_50_labeled))
print(np.sum(mnist_train_y_500_labeled))
print(np.sum(mnist_train_y_10_labeled))
print(np.sum(mnist_train_y_10000_labeled))
print(np.sum(mnist_train_y_all_labeled))


0.0
50.0
500.0
10.0
10000.0
55000.0


In [8]:
"""Define get next batch method"""
current_index = 0
current_data_shuffle_mat = None
def get_next_batch(data, classes, num_to_select):
    global current_index, current_data_shuffle_mat
    #  shuffle with labels
    if current_data_shuffle_mat is None or current_index + num_to_select >= len(data):
        current_data_shuffle_mat =  np.asarray(zip(data, classes))
        np.random.shuffle(current_data_shuffle_mat)
        current_index = 0
    
    data_mat_selected = np.asarray(list(\
                        current_data_shuffle_mat[current_index: current_index + num_to_select,0]))
    data_classes_selected = np.asarray(list(\
                        current_data_shuffle_mat[current_index: current_index + num_to_select,1]))
    current_index += num_to_select
    
#     return data_mat_selected.astype(float), data_classes_selected.astype(float)
    return data_mat_selected, data_classes_selected




#  Import / Clean Sleep Data

In [ ]:
is_5_mins = True

train_original_df = pd.read_csv('/***')
#  choose validation set or test set
valid_original_df = pd.read_csv('/***')


In [10]:
print(train_original_df.info())
train_original_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Columns: 609 entries, time to userId
dtypes: float64(607), int64(1), object(1)
memory usage: 47.1+ MB
None


,time,hr0,hr1,hr2,hr3,hr4,hr5,hr6,hr7,hr8,...,ox298,ox299,Invalid,S4,S3,S2,S1,REM,W,userId
0,0,84.013123,83.012131,82.011139,80.009155,80.009155,81.010147,82.011139,82.011139,83.012131,...,97.001602,98.002594,0.003333,0.0,0.0,0.0,0.0,0.0,0.996667,ins2
1,1,69.010452,70.011444,70.011444,70.011444,70.011444,70.011444,69.010452,70.011444,69.010452,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2
2,2,77.006180,76.005188,77.006180,76.005188,77.006180,77.006180,77.006180,77.006180,78.007172,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2
3,3,75.016403,75.016403,75.016403,76.005188,75.016403,75.016403,75.016403,75.016403,76.005188,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2
4,4,72.013428,72.013428,73.014420,72.013428,73.014420,72.013428,73.014420,73.014420,72.013428,...,97.001602,97.001602,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,ins2


In [11]:
print(valid_original_df.info())
valid_original_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Columns: 609 entries, time to userId
dtypes: float64(607), int64(1), object(1)
memory usage: 5.8+ MB
None


,time,hr0,hr1,hr2,hr3,hr4,hr5,hr6,hr7,hr8,...,ox298,ox299,Invalid,S4,S3,S2,S1,REM,W,userId
0,0,83.076219,82.075227,82.075227,81.074235,81.074235,81.074235,81.074235,81.074235,81.074235,...,95.017929,95.017929,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
1,1,88.068971,88.068971,88.068971,88.068971,87.080186,86.079194,83.076219,81.074235,80.073243,...,94.018463,94.018463,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
2,2,75.083543,75.083543,75.083543,75.083543,74.082551,74.082551,74.082551,73.081559,73.081559,...,93.023575,93.023575,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
3,3,73.081559,74.082551,74.082551,75.083543,75.083543,75.083543,75.083543,74.082551,73.081559,...,93.023575,94.018463,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
4,4,78.071260,78.071260,79.072251,79.072251,79.072251,79.072251,79.072251,79.072251,80.073243,...,95.017929,95.017929,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8


In [12]:
def mean_center_data(passed_df):
    """Takes a train/test/validation set in format time, hr, ox, classes, userId and mean centered the
    data for each user"""
    
    #  find mean ox and hr
    mean_values_by_userId = []

    for name, group in passed_df.groupby('userId'):  
        if is_5_mins:
            mean_hr = group.loc[:, 'hr0':'hr299'].as_matrix().mean()
            mean_ox = group.loc[:, 'ox0':'ox299'].as_matrix().mean()
        else:
            mean_hr = group.loc[:, 'hr0':'hr59'].as_matrix().mean()
            mean_ox = group.loc[:, 'ox0':'ox59'].as_matrix().mean()

        mean_values_by_userId.append([name, mean_hr, mean_ox])
        
        
    #  convert to dataframe
    mean_values_by_userId = pd.DataFrame(mean_values_by_userId)
    mean_values_by_userId.columns = ['userId', 'mean_hr', 'mean_ox']
        
        
    #  merge with training/etc data, then mean center, then drop the mean values columns
    mean_centered_df = pd.merge(passed_df, mean_values_by_userId, on='userId', how='left')
    if is_5_mins:
        mean_centered_df.loc[:, 'hr0':'hr299'] = mean_centered_df.loc[:, 'hr0':'hr299'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_hr'].as_matrix()).T
        mean_centered_df.loc[:, 'ox0':'ox299'] = mean_centered_df.loc[:, 'ox0':'ox299'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_ox'].as_matrix()).T
    else:
        mean_centered_df.loc[:, 'hr0':'hr59'] = mean_centered_df.loc[:, 'hr0':'hr59'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_hr'].as_matrix()).T
        mean_centered_df.loc[:, 'ox0':'ox59'] = mean_centered_df.loc[:, 'ox0':'ox59'].as_matrix() - \
                                                np.atleast_2d(mean_centered_df.loc[:, 'mean_ox'].as_matrix()).T


    mean_centered_df.drop(['mean_hr','mean_ox'], axis=1, inplace=True)
    return mean_centered_df

In [13]:
mean_centered_train_df = mean_center_data(train_original_df)
mean_centered_valid_df = mean_center_data(valid_original_df)

In [14]:
mean_centered_valid_df.head(2)

,time,hr0,hr1,hr2,hr3,hr4,hr5,hr6,hr7,hr8,...,ox298,ox299,Invalid,S4,S3,S2,S1,REM,W,userId
0,0,18.389260,17.388268,17.388268,16.387276,16.387276,16.387276,16.387276,16.387276,16.387276,...,2.978348,2.978348,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8
1,1,23.382012,23.382012,23.382012,23.382012,22.393227,21.392235,18.389260,16.387276,15.386284,...,1.978882,1.978882,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ins8


In [15]:
#  find majority class labels


#  remove class percent columns (6) at the end


#  this section accounts for invalid entries. First it actively saves all valid classes as
#  last_valid_class, then if any valid classes are encountered they are replaced with this 
#  last valid class value
#  all users are linked together so values might be taken from the user before if they start out invalid
#  this should be fairly infrequent though and to boot most users start and end in the wake state so it
#  should be alright to link their values back to back like this



last_valid_class = 0  #  this is the wake class, it's a fair to assume most start in the wake phase

#  majority classes for train
majority_classes_train = []
for row in mean_centered_train_df.iloc[:,-8:-1].as_matrix():
    #  4 classes DEEP(S4/S3), LIGHT(S2/S1), REM, W
#     row = [row[0], row[1] + row[2], row[3] + row[4], row[5], row[6]]
    #  3 classes DEEP, LIGHT (including REM), Wake
    row = [row[0], row[1] + row[2], row[3] + row[4] + row[5], row[6]]
    #  2 classes SLEEP, Wake
#     row = [row[0], row[1] + row[2] + row[3] + row[4] + row[5], row[6]]


    current_class = np.argmax(row) - 1
    if current_class < 0:
        current_class = last_valid_class
    else:
        last_valid_class = current_class
    majority_classes_train.append(current_class)



last_valid_class = 0  #  this is the wake class, it's a fair to assume most start in the wake phase

#  majority classes for train
majority_classes_valid = []
for row in mean_centered_valid_df.iloc[:,-8:-1].as_matrix():
    #  4 classes DEEP(S4/S3), LIGHT(S2/S1), REM, W
#     row = [row[0], row[1] + row[2], row[3] + row[4], row[5], row[6]]
    # 3 classes DEEP, LIGHT (including REM), Wake
    row = [row[0], row[1] + row[2], row[3] + row[4] + row[5], row[6]]
    #  2 classes SLEEP, Wake
#     row = [row[0], row[1] + row[2] + row[3] + row[4] + row[5], row[6]]

    
    current_class = np.argmax(row) - 1
    if current_class < 0:
        current_class = last_valid_class
    else:
        last_valid_class = current_class
    majority_classes_valid.append(current_class)



In [16]:
#  check to make sure conversion worked (should see 3 for d,l,w etc)
np.bincount(majority_classes_train)

array([1800, 6279, 2048])

In [17]:
#  extract stardize shuffle

#  extract data for train/valid

#  include time (in n minute instances) since sleep began
if is_5_mins:
    train_original_mat = mean_centered_train_df.loc[:,'time':'ox299'].as_matrix()
    valid_original_mat = mean_centered_valid_df.loc[:,'time':'ox299'].as_matrix()
else:
    train_original_mat = mean_centered_train_df.loc[:,'time':'ox59'].as_matrix()
    valid_original_mat = mean_centered_valid_df.loc[:,'time':'ox59'].as_matrix()
# train_original_mat = mean_centered_train_df.loc[:,'hr0':'ox299'].as_matrix()
# valid_original_mat = mean_centered_valid_df.loc[:,'hr0':'ox299'].as_matrix()

train_original_mat

array([[  0.        ,  13.26134914,  12.26035714, ...,   0.79259893,
          0.79259893,   1.79359093],
       [  1.        ,  -1.74132186,  -0.74032986, ...,   0.79259893,
          0.79259893,   0.79259893],
       [  2.        ,   6.25440614,   5.25341414, ...,   0.79259893,
          0.79259893,   0.79259893],
       ..., 
       [ 93.        ,  10.38672983,  10.38672983, ...,   1.5138803 ,
          1.5138803 ,   1.5138803 ],
       [ 94.        ,   1.54526583,   0.63063183, ...,   0.6602223 ,
          0.6602223 ,   0.6602223 ],
       [ 95.        ,   9.47209483,  10.08185083, ...,   0.5382703 ,
          0.6602223 ,   0.6602223 ]])

In [18]:
#  standardize
scaler = StandardScaler()
scaler.fit(train_original_mat)
train_std_mat = scaler.transform(train_original_mat)
valid_std_mat = scaler.transform(valid_original_mat)

In [19]:
#  shuffle with labels
train_shuffled_mat =  np.asarray(zip(train_std_mat, majority_classes_train))
np.random.shuffle(train_shuffled_mat)
valid_shuffled_mat =  np.asarray(zip(valid_std_mat, majority_classes_valid))
np.random.shuffle(valid_shuffled_mat)

train_mat = np.asarray(list(train_shuffled_mat[:,0]))
train_classes = train_shuffled_mat[:,1]
valid_mat = np.asarray(list(valid_shuffled_mat[:,0]))
valid_classes = valid_shuffled_mat[:,1]

In [20]:
#  one hot encoded classes in case I need them

#  TODO:  change this for OHE 6 , 4, then 3, 2

train_classes_ohe = np.zeros((len(train_classes),3))
for i,c in enumerate(train_classes):
    train_classes_ohe[i,c] = 1.0
    
valid_classes_ohe = np.zeros((len(valid_classes),3))
for i,c in enumerate(valid_classes):
    valid_classes_ohe[i,c] = 1.0

In [21]:
train_classes_ohe

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       ..., 
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

In [22]:
train_std_mat.shape

(10127, 601)

In [23]:
sleep_train_y_0_labeled = keep_n_labels(train_classes_ohe, 0)
sleep_train_y_10_labeled = keep_n_labels(train_classes_ohe, 10)
sleep_train_y_50_labeled = keep_n_labels(train_classes_ohe, 50)
sleep_train_y_100_labeled = keep_n_labels(train_classes_ohe, 100)
sleep_train_y_200_labeled = keep_n_labels(train_classes_ohe, 200)
sleep_train_y_500_labeled = keep_n_labels(train_classes_ohe, 500)
sleep_train_y_1000_labeled = keep_n_labels(train_classes_ohe, 1000)
sleep_train_y_5000_labeled = keep_n_labels(train_classes_ohe, 5000)
sleep_train_y_10000_labeled = keep_n_labels(train_classes_ohe, 10000)
sleep_train_y_all_labeled = np.copy(train_classes_ohe)



print(np.sum(sleep_train_y_0_labeled))
print(np.sum(sleep_train_y_50_labeled))
print(np.sum(sleep_train_y_500_labeled))
print(np.sum(sleep_train_y_10_labeled))
print(np.sum(sleep_train_y_10000_labeled))
print(np.sum(sleep_train_y_all_labeled))


0.0
50.0
500.0
10.0
10000.0
10127.0


#  Model SLEEP

In [24]:
sess = tf.InteractiveSession()    
   
def relu(passed_mat):
    return tf.maximum(0.0, passed_mat)

def softmax(passed_mat):
    return tf.exp(passed_mat) / tf.reduce_sum(tf.exp(passed_mat))





#  batch norm without scale or offset
def batch_norm(passed_mat, isTrain, isUncorrupted):
    e = 0.00001
    decay = 0.999


    
    
    #  track this with moving averages in order to make predictions on the fly
    pop_mean = tf.Variable(tf.zeros([passed_mat.get_shape()[-1]]), trainable=False)
    pop_var = tf.Variable(tf.ones([passed_mat.get_shape()[-1]]), trainable=False)

    
    
#     if isTrain:
#     if Tensor.eval(training):
    def training_batch_norn():
        mean, var = tf.nn.moments(passed_mat, axes=[0])
        #  use moving average to estimate population averages on the fly
        
        if isUncorrupted:
            train_mean = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
            train_var = tf.assign(pop_var, pop_var * decay + var * (1 - decay))
        else:
            train_mean = mean
            train_var = var
        
        #  this makes sure that the pop weights are updated before batch_norm executes
        with tf.control_dependencies([train_mean, train_var]):
            return _batch_norm_op(passed_mat, mean, var, e)

    
    def predictive_batch_norm():
        #  use population mean and variance instead of batch values
        return _batch_norm_op(passed_mat, pop_mean, pop_var, e)




    return tf.cond(isTrain, training_batch_norn, predictive_batch_norm)




def _batch_norm_op(passed_mat, mean, var, e):
    #  helps prevent dividing by 0
    var_scale = tf.rsqrt(var + e)
    bn_out = (passed_mat * var_scale) - (mean * var_scale)
    return bn_out






#  input data and targets
x = tf.placeholder(tf.float32, [None, 601], name="x")
y_ = tf.placeholder(tf.float32, [None, 3], name="y_")
isTrain = tf.placeholder(tf.bool, shape=(), name="isTrain")
isLabeled = tf.placeholder(tf.bool, shape=(), name="isLabeled")
batch_size = 50


with tf.name_scope('LadderNetwork') as scope:

    
    
    
    
    
    with tf.name_scope('UncorruptedForwardPass'):


        #  layer 1
        with tf.name_scope('Layer1'):
            w1_dim_1, w1_dim_2 = 601, 400
            W1 = tf.Variable(tf.random_normal([w1_dim_1, w1_dim_2], stddev=0.001))
            b1 = tf.Variable(tf.random_normal([w1_dim_2], stddev=0.001))
            z1 = batch_norm(tf.matmul(x, W1) + b1, isTrain, True)
            #  these will be trained and adjusted with all other variables
            scale1 = tf.Variable(tf.ones([z1.get_shape()[-1]]))
            beta1 = tf.Variable(tf.zeros([z1.get_shape()[-1]]))
            h1 = relu(scale1 * z1 + beta1)


        #  layer 2
        with tf.name_scope('Layer2'):
            w2_dim_1, w2_dim_2 = 400, 200
            W2 = tf.Variable(tf.random_normal([w2_dim_1, w2_dim_2], stddev=0.001))
            b2 = tf.Variable(tf.random_normal([w2_dim_2], stddev=0.001))
            z2 = batch_norm(tf.matmul(h1, W2) + b2, isTrain, True)
            #  these will be trained and adjusted with all other variables
            scale2 = tf.Variable(tf.ones([z2.get_shape()[-1]]))
            beta2 = tf.Variable(tf.zeros([z2.get_shape()[-1]]))
            h2 = relu(scale2 * z2 + beta2)


        #  layer 3
        with tf.name_scope('Layer3'):
            w3_dim_1, w3_dim_2 = 200, 3
            W3 = tf.Variable(tf.random_normal([w3_dim_1, w3_dim_2], stddev=0.001))
            b3 = tf.Variable(tf.random_normal([w3_dim_2], stddev=0.001))
            z3 = batch_norm(tf.matmul(h2, W3) + b3, isTrain, True)
            #  these will be trained and adjusted with all other variables
            scale3 = tf.Variable(tf.ones([z3.get_shape()[-1]]))
            beta3 = tf.Variable(tf.zeros([z3.get_shape()[-1]]))
            h3 = softmax(scale3 * z3 + beta3)
            y = h3


    
    
    
    
    
    with tf.name_scope('CorruptedForwardPass'):

        #  _n stands for noisy versions in the corrupted forward pass
        true_const_tensor = tf.constant(True)
        std = 0.2
        
        
        noise0 = tf.random_normal(shape=tf.shape(x), mean=0.0, stddev=std, dtype=tf.float32) 
        x_n = h0_n = x + noise0
        
        
        #  layer 1
        with tf.name_scope('Layer1'):
            z1_n_pre = tf.matmul(h0_n, W1) + b1
            noise1 = tf.random_normal(shape=tf.shape(z1_n_pre), mean=0.0, stddev=std, dtype=tf.float32) 
            z1_n = batch_norm(z1_n_pre, true_const_tensor, False) + noise1
            h1_n = relu(scale1 * z1 + beta1)


        #  layer 2
        with tf.name_scope('Layer2'):
            z2_n_pre = tf.matmul(h1_n, W2) + b2
            noise2 = tf.random_normal(shape=tf.shape(z2_n_pre), mean=0.0, stddev=std, dtype=tf.float32) 
            z2_n = batch_norm(z2_n_pre, true_const_tensor, False) + noise2
            h2_n = relu(scale2 * z2 + beta2)


        #  layer 3
        with tf.name_scope('Layer3'):
            z3_n_pre = tf.matmul(h2_n, W3) + b3
            noise3 = tf.random_normal(shape=tf.shape(z3_n_pre), mean=0.0, stddev=std, dtype=tf.float32) 
            z3_n = batch_norm(z3_n_pre, true_const_tensor, False) + noise3
            h3_n = softmax(scale3 * z3 + beta3)
            y_n = h3_n

            
            


    with tf.name_scope('DenoisingBackwardsPass'):


        def g_gauss(z_c, u, size):
            "gaussian denoising function proposed in the original paper"
            wi = lambda inits, name: tf.Variable(inits * tf.ones([size]), name=name)
            a1 = wi(0., 'a1')
            a2 = wi(1., 'a2')
            a3 = wi(0., 'a3')
            a4 = wi(0., 'a4')
            a5 = wi(0., 'a5')

            a6 = wi(0., 'a6')
            a7 = wi(1., 'a7')
            a8 = wi(0., 'a8')
            a9 = wi(0., 'a9')
            a10 = wi(0., 'a10')

            mu = a1 * tf.sigmoid(a2 * u + a3) + a4 * u + a5
            v = a6 * tf.sigmoid(a7 * u + a8) + a9 * u + a10

            z_est = (z_c - mu) * v + mu
            return z_est



        d_cost = []  # to store the denoising cost of all layers

        #  layer 3
        with tf.name_scope('Layer3'):
            
            #  this first layer doesn not downsample but measures the loss of of output as compared
            #  to it's noisey version and it's batch normed counter part
            z = z3
            z_c = z3_n

            u = y_n
            #  TODO:  put u_bn back to batch norm
            u_bn = y_n
#             u_bn = batch_norm(u, isTrain, False)
            z3_dn = g_gauss(z_c, u_bn, w3_dim_2)
            z3_dn_bn = batch_norm(z3_dn, isTrain, False)
    
            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z3_dn_bn), 1)) / w3_dim_2) * 0.10) 
    
    

        #  layer 2
        with tf.name_scope('Layer2'):
            
            V2 = tf.Variable(tf.random_normal([w3_dim_2, w3_dim_1], stddev=0.001))
            u_bn = batch_norm(tf.matmul(z3_dn, V2), isTrain, False)  
            
            z = z2
            z_c = z2_n
            z2_dn = g_gauss(z_c, u_bn, w2_dim_2)
            z2_dn_bn = batch_norm(z2_dn, isTrain, False)

            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z2_dn_bn), 1)) / w2_dim_2) * 0.10)
           
              
        
        #  layer 1
        with tf.name_scope('Layer1'):
                
            V1 = tf.Variable(tf.random_normal([w2_dim_2, w2_dim_1], stddev=0.001))
            u_bn = batch_norm(tf.matmul(z2_dn, V1), isTrain, False) 
                       
            z = z1
            z_c = z1_n
            z1_dn = g_gauss(z_c, u_bn, w1_dim_2)
            z1_dn_bn = batch_norm(z1_dn, isTrain, False)  #  batch norm the denoised z for the d_cost

            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z1_dn_bn), 1)) / w1_dim_2) * 10.0)
  
            
                   
        #  layer 0
        with tf.name_scope('Layer0'): 
                
            V0 = tf.Variable(tf.random_normal([w1_dim_2, w1_dim_1], stddev=0.001))
            u_bn = batch_norm(tf.matmul(z1_dn, V0), isTrain, False) 
                       
            z = x
            z_c = x_n
            z0_dn = g_gauss(z_c, u_bn, w1_dim_1)
            z0_dn_bn = batch_norm(z0_dn, isTrain, False)

            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z0_dn_bn), 1)) / w1_dim_1) * 1000.0)

        x_dn = z0_dn
    
    
    









    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    




with tf.name_scope('SupervisedLoss') as scope:
    def cross_ent():
        return tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_n), reduction_indices=[1]))
    def set_to_zero():
        return tf.constant(0.0)
    cross_entropy = tf.cond(isLabeled, cross_ent, set_to_zero)
#     cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_n), reduction_indices=[1]))

    

with tf.name_scope('UnupervisedLoss') as scope:
    # calculate total unsupervised cost by adding the denoising cost of all layers
    unsupervised_cost = tf.add_n(d_cost)
    loss = cross_entropy + unsupervised_cost  # total cost



    
with tf.name_scope('Optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
#     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    
with tf.name_scope('Accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    



#  summary writers for debugging
summary_writer = tf.summary.FileWriter( \
 '/home/tuna/Projects/CS678/Ladder_Networks_Project_1/tf_logs/ln_sleep_100_labels/v1',\
                                    graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', loss )
merged_summary_op = tf.summary.merge_all()
#  additional for seperate runs
valid_acc_summary = tf.summary.scalar( 'validation_accuracy', accuracy )
test_acc_summary = tf.summary.scalar( 'test_accuracy', accuracy )

sess.run(tf.global_variables_initializer())



for i in range(10000):
#     batch = mnist.train.next_batch(batch_size)
    batch_xs, batch_ys = get_next_batch(train_mat, sleep_train_y_10_labeled, batch_size)

#     if i < 1000:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys,
#                                                                           isTrain: True, isLabeled: True})
#     else:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: np.zeros(batch_ys.shape, dtype=np.float32),
#                                                                           isTrain: True, isLabeled: True})
#     _, summary_ = sess.run([train_step, merged_summary_op], feed_dict={x: batch[0], y_: np.zeros(batch[1].shape, dtype=np.float32),
#                                                                       isTrain: True, isLabeled: False})
    _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys, isTrain: True, isLabeled: True})
    

    summary_writer.add_summary(summary_, i)
    if i%100 == 0:
        #  print training loss
        print("step %d, train_loss %g" % (i, train_loss))
        
        #  validation set results
        acc, ls, valid_summary = sess.run([accuracy, loss, valid_acc_summary], \
                                        feed_dict={x: valid_mat, y_: valid_classes_ohe, isTrain: False, isLabeled: True})
        summary_writer.add_summary(valid_summary, i)
        print("step %d, validation accuracy %g, loss %g" % (i, acc, ls))

        
#  test set results
acc, test_acc_summary = sess.run([accuracy, test_acc_summary], feed_dict={x: valid_mat, y_: valid_classes_ohe, isTrain: False, isLabeled: True})
summary_writer.add_summary(test_acc_summary, 1)
summary_writer.add_summary(test_acc_summary, 100)
summary_writer.add_summary(test_acc_summary, 1000)
summary_writer.add_summary(test_acc_summary, 10)
print("test accuracy %g" % acc)



summary_writer.close()
sess.close()

step 0, train_loss 599.511
step 0, validation accuracy 0.260974, loss 711.043
step 100, train_loss 65.1258
step 100, validation accuracy 0.177973, loss 706.467
step 200, train_loss 94.8783
step 200, validation accuracy 0.178771, loss 706.178
step 300, train_loss 209.753
step 300, validation accuracy 0.190742, loss 706.343
step 400, train_loss 59.3607
step 400, validation accuracy 0.189146, loss 706.424
step 500, train_loss 173.361
step 500, validation accuracy 0.185954, loss 706.61
step 600, train_loss 57.2304
step 600, validation accuracy 0.186752, loss 706.88
step 700, train_loss 162.288
step 700, validation accuracy 0.18755, loss 707.088
step 800, train_loss 212.252
step 800, validation accuracy 0.18755, loss 707.345
step 900, train_loss 203.412
step 900, validation accuracy 0.185954, loss 707.512
step 1000, train_loss 83.6336
step 1000, validation accuracy 0.186752, loss 707.905
step 1100, train_loss 482.394
step 1100, validation accuracy 0.185954, loss 708.095
step 1200, train_los

In [9]:
sess = tf.InteractiveSession()    
   
def relu(passed_mat):
    return tf.maximum(0.0, passed_mat)

def softmax(passed_mat):
    return tf.exp(passed_mat) / tf.reduce_sum(tf.exp(passed_mat))





#  batch norm without scale or offset
def batch_norm(passed_mat, isTrain, isUncorrupted):
    e = 0.00001
    decay = 0.999


    
    
    #  track this with moving averages in order to make predictions on the fly
    pop_mean = tf.Variable(tf.zeros([passed_mat.get_shape()[-1]]), trainable=False)
    pop_var = tf.Variable(tf.ones([passed_mat.get_shape()[-1]]), trainable=False)

    
    
#     if isTrain:
#     if Tensor.eval(training):
    def training_batch_norn():
        mean, var = tf.nn.moments(passed_mat, axes=[0])
        #  use moving average to estimate population averages on the fly
        
        if isUncorrupted:
            train_mean = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
            train_var = tf.assign(pop_var, pop_var * decay + var * (1 - decay))
        else:
            train_mean = mean
            train_var = var
        
        #  this makes sure that the pop weights are updated before batch_norm executes
        with tf.control_dependencies([train_mean, train_var]):
            return _batch_norm_op(passed_mat, mean, var, e)

    
    def predictive_batch_norm():
        #  use population mean and variance instead of batch values
        return _batch_norm_op(passed_mat, pop_mean, pop_var, e)




    return tf.cond(isTrain, training_batch_norn, predictive_batch_norm)




def _batch_norm_op(passed_mat, mean, var, e):
    #  helps prevent dividing by 0
    var_scale = tf.rsqrt(var + e)
    bn_out = (passed_mat * var_scale) - (mean * var_scale)
    return bn_out






#  input data and targets
x = tf.placeholder(tf.float32, [None, 784], name="x")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")
isTrain = tf.placeholder(tf.bool, shape=(), name="isTrain")
isLabeled = tf.placeholder(tf.bool, shape=(), name="isLabeled")
batch_size = 50


with tf.name_scope('LadderNetwork') as scope:

    
    
    
    
    
    with tf.name_scope('UncorruptedForwardPass'):


        #  layer 1
        with tf.name_scope('Layer1'):
            w1_dim_1, w1_dim_2 = 784, 400
            W1 = tf.Variable(tf.random_normal([w1_dim_1, w1_dim_2], stddev=0.001))
            b1 = tf.Variable(tf.random_normal([w1_dim_2], stddev=0.001))
            z1 = batch_norm(tf.matmul(x, W1) + b1, isTrain, True)
            #  these will be trained and adjusted with all other variables
            scale1 = tf.Variable(tf.ones([z1.get_shape()[-1]]))
            beta1 = tf.Variable(tf.zeros([z1.get_shape()[-1]]))
            h1 = relu(scale1 * z1 + beta1)


        #  layer 2
        with tf.name_scope('Layer2'):
            w2_dim_1, w2_dim_2 = 400, 200
            W2 = tf.Variable(tf.random_normal([w2_dim_1, w2_dim_2], stddev=0.001))
            b2 = tf.Variable(tf.random_normal([w2_dim_2], stddev=0.001))
            z2 = batch_norm(tf.matmul(h1, W2) + b2, isTrain, True)
            #  these will be trained and adjusted with all other variables
            scale2 = tf.Variable(tf.ones([z2.get_shape()[-1]]))
            beta2 = tf.Variable(tf.zeros([z2.get_shape()[-1]]))
            h2 = relu(scale2 * z2 + beta2)


        #  layer 3
        with tf.name_scope('Layer3'):
            w3_dim_1, w3_dim_2 = 200, 10
            W3 = tf.Variable(tf.random_normal([w3_dim_1, w3_dim_2], stddev=0.001))
            b3 = tf.Variable(tf.random_normal([w3_dim_2], stddev=0.001))
            z3 = batch_norm(tf.matmul(h2, W3) + b3, isTrain, True)
            #  these will be trained and adjusted with all other variables
            scale3 = tf.Variable(tf.ones([z3.get_shape()[-1]]))
            beta3 = tf.Variable(tf.zeros([z3.get_shape()[-1]]))
            h3 = softmax(scale3 * z3 + beta3)
            y = h3


    
    
    
    
    
    with tf.name_scope('CorruptedForwardPass'):

        #  _n stands for noisy versions in the corrupted forward pass
        true_const_tensor = tf.constant(True)
        std = 0.2
        
        
        noise0 = tf.random_normal(shape=tf.shape(x), mean=0.0, stddev=std, dtype=tf.float32) 
        x_n = h0_n = x + noise0
        
        
        #  layer 1
        with tf.name_scope('Layer1'):
            z1_n_pre = tf.matmul(h0_n, W1) + b1
            noise1 = tf.random_normal(shape=tf.shape(z1_n_pre), mean=0.0, stddev=std, dtype=tf.float32) 
            z1_n = batch_norm(z1_n_pre, true_const_tensor, False) + noise1
            h1_n = relu(scale1 * z1 + beta1)


        #  layer 2
        with tf.name_scope('Layer2'):
            z2_n_pre = tf.matmul(h1_n, W2) + b2
            noise2 = tf.random_normal(shape=tf.shape(z2_n_pre), mean=0.0, stddev=std, dtype=tf.float32) 
            z2_n = batch_norm(z2_n_pre, true_const_tensor, False) + noise2
            h2_n = relu(scale2 * z2 + beta2)


        #  layer 3
        with tf.name_scope('Layer3'):
            z3_n_pre = tf.matmul(h2_n, W3) + b3
            noise3 = tf.random_normal(shape=tf.shape(z3_n_pre), mean=0.0, stddev=std, dtype=tf.float32) 
            z3_n = batch_norm(z3_n_pre, true_const_tensor, False) + noise3
            h3_n = softmax(scale3 * z3 + beta3)
            y_n = h3_n

            
            


    with tf.name_scope('DenoisingBackwardsPass'):


        def g_gauss(z_c, u, size):
            "gaussian denoising function proposed in the original paper"
            wi = lambda inits, name: tf.Variable(inits * tf.ones([size]), name=name)
            a1 = wi(0., 'a1')
            a2 = wi(1., 'a2')
            a3 = wi(0., 'a3')
            a4 = wi(0., 'a4')
            a5 = wi(0., 'a5')

            a6 = wi(0., 'a6')
            a7 = wi(1., 'a7')
            a8 = wi(0., 'a8')
            a9 = wi(0., 'a9')
            a10 = wi(0., 'a10')

            mu = a1 * tf.sigmoid(a2 * u + a3) + a4 * u + a5
            v = a6 * tf.sigmoid(a7 * u + a8) + a9 * u + a10

            z_est = (z_c - mu) * v + mu
            return z_est



        d_cost = []  # to store the denoising cost of all layers

        #  layer 3
        with tf.name_scope('Layer3'):
            
            #  this first layer doesn not downsample but measures the loss of of output as compared
            #  to it's noisey version and it's batch normed counter part
            z = z3
            z_c = z3_n

            u = y_n
            #  TODO:  put u_bn back to batch norm
            u_bn = y_n
#             u_bn = batch_norm(u, isTrain, False)
            z3_dn = g_gauss(z_c, u_bn, w3_dim_2)
            z3_dn_bn = batch_norm(z3_dn, isTrain, False)
    
            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z3_dn_bn), 1)) / w3_dim_2) * 0.10) 
    
    

        #  layer 2
        with tf.name_scope('Layer2'):
            
            V2 = tf.Variable(tf.random_normal([w3_dim_2, w3_dim_1], stddev=0.001))
            u_bn = batch_norm(tf.matmul(z3_dn, V2), isTrain, False)  
            
            z = z2
            z_c = z2_n
            z2_dn = g_gauss(z_c, u_bn, w2_dim_2)
            z2_dn_bn = batch_norm(z2_dn, isTrain, False)

            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z2_dn_bn), 1)) / w2_dim_2) * 0.10)
           
              
        
        #  layer 1
        with tf.name_scope('Layer1'):
                
            V1 = tf.Variable(tf.random_normal([w2_dim_2, w2_dim_1], stddev=0.001))
            u_bn = batch_norm(tf.matmul(z2_dn, V1), isTrain, False) 
                       
            z = z1
            z_c = z1_n
            z1_dn = g_gauss(z_c, u_bn, w1_dim_2)
            z1_dn_bn = batch_norm(z1_dn, isTrain, False)  #  batch norm the denoised z for the d_cost

            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z1_dn_bn), 1)) / w1_dim_2) * 10.0)
  
            
                   
        #  layer 0
        with tf.name_scope('Layer0'): 
                
            V0 = tf.Variable(tf.random_normal([w1_dim_2, w1_dim_1], stddev=0.001))
            u_bn = batch_norm(tf.matmul(z1_dn, V0), isTrain, False) 
                       
            z = x
            z_c = x_n
            z0_dn = g_gauss(z_c, u_bn, w1_dim_1)
            z0_dn_bn = batch_norm(z0_dn, isTrain, False)

            # append the cost of this layer to d_cost
            d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z - z0_dn_bn), 1)) / w1_dim_1) * 1000.0)

        x_dn = z0_dn
    
    
    









    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    




with tf.name_scope('SupervisedLoss') as scope:
    def cross_ent():
        return tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_n), reduction_indices=[1]))
    def set_to_zero():
        return tf.constant(0.0)
    cross_entropy = tf.cond(isLabeled, cross_ent, set_to_zero)
#     cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_n), reduction_indices=[1]))

    

with tf.name_scope('UnupervisedLoss') as scope:
    # calculate total unsupervised cost by adding the denoising cost of all layers
    unsupervised_cost = tf.add_n(d_cost)
    loss = cross_entropy + unsupervised_cost  # total cost



    
with tf.name_scope('Optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
#     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    
with tf.name_scope('Accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    



#  summary writers for debugging
summary_writer = tf.summary.FileWriter( \
        '/home/tuna/Projects/CS678/Ladder_Networks_Project_1/tf_logs/ln_mnist_5000_labels/v6', graph=sess.graph )
acc_summary = tf.summary.scalar( 'accuracy', accuracy )
loss_summary = tf.summary.scalar( 'loss', loss )
merged_summary_op = tf.summary.merge_all()
#  additional for seperate runs
valid_acc_summary = tf.summary.scalar( 'validation_accuracy', accuracy )
test_acc_summary = tf.summary.scalar( 'test_accuracy', accuracy )

sess.run(tf.global_variables_initializer())



for i in range(15000):
#     batch = mnist.train.next_batch(batch_size)
    batch_xs, batch_ys = get_next_batch(mnist.train.images, mnist_train_y_5000_labeled, batch_size)

#     if i < 1000:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys,
#                                                                           isTrain: True, isLabeled: True})
#     else:
#         _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: np.zeros(batch_ys.shape, dtype=np.float32),
#                                                                           isTrain: True, isLabeled: True})
#     _, summary_ = sess.run([train_step, merged_summary_op], feed_dict={x: batch[0], y_: np.zeros(batch[1].shape, dtype=np.float32),
#                                                                       isTrain: True, isLabeled: False})
    _, summary_, train_loss = sess.run([train_step, merged_summary_op, loss], feed_dict={x: batch_xs, y_: batch_ys,
                                                                      isTrain: True, isLabeled: True})
    

    summary_writer.add_summary(summary_, i)
    if i%100 == 0:
        #  print training loss
        print("step %d, train_loss %g" % (i, train_loss))
        
        #  validation set results
        acc, ls, valid_summary = sess.run([accuracy, loss, valid_acc_summary], \
                                        feed_dict={x: mnist.validation.images, y_: mnist.validation.labels,\
                                                  isTrain: False, isLabeled: True})
        summary_writer.add_summary(valid_summary, i)
        print("step %d, validation accuracy %g, loss %g" % (i, acc, ls))

        
#  test set results
acc, test_acc_summary = sess.run([accuracy, test_acc_summary], feed_dict={x: mnist.test.images, y_: mnist.test.labels,\
                                                          isTrain: False, isLabeled: True})
summary_writer.add_summary(test_acc_summary, 1)
summary_writer.add_summary(test_acc_summary, 100)
summary_writer.add_summary(test_acc_summary, 1000)
summary_writer.add_summary(test_acc_summary, 10)
print("test accuracy %g" % acc)



summary_writer.close()
sess.close()

step 0, train_loss 127.136
step 0, validation accuracy 0.0924, loss 122.314
step 100, train_loss 53.0978
step 100, validation accuracy 0.11, loss 121.876
step 200, train_loss 56.4439
step 200, validation accuracy 0.0924, loss 121.899
step 300, train_loss 58.4609
step 300, validation accuracy 0.0924, loss 121.87
step 400, train_loss 48.1426
step 400, validation accuracy 0.0924, loss 121.812
step 500, train_loss 48.8902
step 500, validation accuracy 0.0924, loss 121.755
step 600, train_loss 49.6925
step 600, validation accuracy 0.0924, loss 121.701
step 700, train_loss 52.6471
step 700, validation accuracy 0.0924, loss 121.63
step 800, train_loss 51.6703
step 800, validation accuracy 0.0924, loss 121.559
step 900, train_loss 48.8234
step 900, validation accuracy 0.0924, loss 121.492
step 1000, train_loss 56.299
step 1000, validation accuracy 0.0924, loss 121.418
step 1100, train_loss 52.5355
step 1100, validation accuracy 0.0924, loss 121.353
step 1200, train_loss 47.0186
step 1200, vali

# Train

In [7]:


# # this is a simpler version of Tensorflow's 'official' version. See:
# # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/layers/python/layers/layers.py#L102
# def batch_norm_wrapper(inputs, isTrain, decay = 0.999):

#     scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
#     beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
#     pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
#     pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)

    
#     epsilon = 0.00001
    
    
#     def training_batch_norm():
#         batch_mean, batch_var = tf.nn.moments(inputs,[0])
#         train_mean = tf.assign(pop_mean,
#                                pop_mean * decay + batch_mean * (1 - decay))
#         train_var = tf.assign(pop_var,
#                               pop_var * decay + batch_var * (1 - decay))
#         with tf.control_dependencies([train_mean, train_var]):
#             return batch_norm_op(inputs,
#                 batch_mean, batch_var, beta, scale, epsilon)
    
#     def predictive_batch_norm():
#         return batch_norm_op(inputs,
#             pop_mean, pop_var, beta, scale, epsilon)

#     return tf.cond(isTrain, training_batch_norm, predictive_batch_norm)

In [ ]:
def batch_normalization(x,
                        mean,
                        variance,
                        offset,
                        scale,
                        variance_epsilon,
                        name=None):
  r"""Batch normalization.
  As described in http://arxiv.org/abs/1502.03167.
  Normalizes a tensor by `mean` and `variance`, and applies (optionally) a
  `scale` \\(\gamma\\) to it, as well as an `offset` \\(\beta\\):
  \\(\frac{\gamma(x-\mu)}{\sigma}+\beta\\)
  `mean`, `variance`, `offset` and `scale` are all expected to be of one of two
  shapes:
    * In all generality, they can have the same number of dimensions as the
      input `x`, with identical sizes as `x` for the dimensions that are not
      normalized over (the 'depth' dimension(s)), and dimension 1 for the
      others which are being normalized over.
      `mean` and `variance` in this case would typically be the outputs of
      `tf.nn.moments(..., keep_dims=True)` during training, or running averages
      thereof during inference.
    * In the common case where the 'depth' dimension is the last dimension in
      the input tensor `x`, they may be one dimensional tensors of the same
      size as the 'depth' dimension.
      This is the case for example for the common `[batch, depth]` layout of
      fully-connected layers, and `[batch, height, width, depth]` for
      convolutions.
      `mean` and `variance` in this case would typically be the outputs of
      `tf.nn.moments(..., keep_dims=False)` during training, or running averages
      thereof during inference.
  Args:
    x: Input `Tensor` of arbitrary dimensionality.
    mean: A mean `Tensor`.
    variance: A variance `Tensor`.
    offset: An offset `Tensor`, often denoted \\(\beta\\) in equations, or
      None. If present, will be added to the normalized tensor.
    scale: A scale `Tensor`, often denoted \\(\gamma\\) in equations, or
      `None`. If present, the scale is applied to the normalized tensor.
    variance_epsilon: A small float number to avoid dividing by 0.
    name: A name for this operation (optional).
  Returns:
    the normalized, scaled, offset tensor.
  """
  with ops.name_scope(name, "batchnorm", [x, mean, variance, scale, offset]):
    inv = math_ops.rsqrt(variance + variance_epsilon)
    if scale is not None:
      inv *= scale
    return x * inv + (offset - mean * inv
                      if offset is not None else -mean * inv)

In [ ]:
    
    
    
    
#     with tf.name_scope('DenoisingBackwardsPass'):


            
#         #  TODO: I think I'm missing something big here... each z_n is only dependant on it's
#         #  realted z_n_i? I'm just using the entire layer on both sides... they say I should
#         #  seperate these things and make them independant?
        
        

#         #  layer 3
#         with tf.name_scope('Layer3'):
            
            
#             V3 = tf.Variable(tf.random_normal([w3_dim_2, w3_dim_1], stddev=0.001))
# #             u3 = batch_norm(V3 * z4_dn)  #  I'll prob need to change this to y in the future
#             u3 = y_n

            
#             a3_1 = tf.Variable(tf.random_normal([batch_size, w3_dim_2], stddev=0.001))
#             a3_2 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_3 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_4 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_5 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             mu_3 = a3_1 * tf.sigmoid(a3_2 * u3 + a3_3) + a3_4 * u3 + a3_5
                        
#             a3_6 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_7 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_8 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_9 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             a3_10 = tf.Variable(tf.random_normal(tf.shape(u3), stddev=0.001), validate_shape=False)
#             v_3 = a3_6 * tf.sigmoid(a3_7 * u3 + a3_8) + a3_9 * u3 + a3_10

#             z3_dn = (z3_n - mu_3) * v_3 + mu_3
    
    
    
    
    

#         #  layer 2
#         with tf.name_scope('Layer2'):
            
             
#             V2 = tf.Variable(tf.random_normal([w2_dim_2, w2_dim_1], stddev=0.001))
#             u2 = batch_norm(V2 * z3_dn, isTrain, True)  #  I'll prob need to change this to y in the future

            
#             a2_1 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_2 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_3 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_4 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_5 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             mu_2 = a2_1 * tf.sigmoid(a2_2 * u2 + a2_3) + a2_4 * u2 + a2_5
                        
#             a2_6 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_7 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_8 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_9 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             a2_10 = tf.Variable(tf.random_normal(tf.shape(u2), stddev=0.001), validate_shape=False)
#             v_2 = a2_6 * tf.sigmoid(a2_7 * u2 + a2_8) + a2_9 * u2 + a2_10

#             z2_dn = (z2_n - mu_2) * v_2 + mu_2
    
    
    
    
    
           
            
            
            
            
            
            
            
            
        
#         #  layer 1
#         with tf.name_scope('Layer1'):
                
                
#             V1 = tf.Variable(tf.random_normal([W1.get_shape()[1], W1.get_shape()[0]], stddev=0.001))
#             u1 = batch_norm(V1 * z1_n)  #  I'll prob need to change this to y in the future

            
#             a1_1 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_2 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_3 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_4 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_5 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             mu_1 = a1_1 * tf.sigmoid(a1_2 * u1 + a1_3) + a1_4 * u1 + a1_5
                        
#             a1_6 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_7 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_8 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_9 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             a1_10 = tf.Variable(tf.random_normal([u1.get_shape()], stddev=0.001))
#             v_1 = a1_6 * tf.sigmoid(a1_7 * u1 + a1_8) + a1_9 * u1 + a1_10

#             z1_dn = (z1_n - mu_1) * v_1 + mu_1
    
#             #  TODO might be incorrect, are there more transformations to get from
#             #  z1 to z0 to x_dn ?
#             x_dn = z1_dn
    
    
    
    

    
    
    
    
    
    
    
    
# print "=== Decoder ==="


# def g_gauss(z_c, u, size):
#     "gaussian denoising function proposed in the original paper"
#     wi = lambda inits, name: tf.Variable(inits * tf.ones([size]), name=name)
#     a1 = wi(0., 'a1')
#     a2 = wi(1., 'a2')
#     a3 = wi(0., 'a3')
#     a4 = wi(0., 'a4')
#     a5 = wi(0., 'a5')

#     a6 = wi(0., 'a6')
#     a7 = wi(1., 'a7')
#     a8 = wi(0., 'a8')
#     a9 = wi(0., 'a9')
#     a10 = wi(0., 'a10')

#     mu = a1 * tf.sigmoid(a2 * u + a3) + a4 * u + a5
#     v = a6 * tf.sigmoid(a7 * u + a8) + a9 * u + a10

#     z_est = (z_c - mu) * v + mu
#     return z_est

# # Decoder
# z_est = {}
# d_cost = []  # to store the denoising cost of all layers
# for l in range(L, -1, -1):
#     print "Layer ", l, ": ", layer_sizes[l+1] if l+1 < len(layer_sizes) else None, " -> ", layer_sizes[l], ", denoising cost: ", denoising_cost[l]
#     z, z_c = clean['unlabeled']['z'][l], corr['unlabeled']['z'][l]
#     m, v = clean['unlabeled']['m'].get(l, 0), clean['unlabeled']['v'].get(l, 1-1e-10)
#     if l == L:
#         u = unlabeled(y_c)
#     else:
#         u = tf.matmul(z_est[l+1], weights['V'][l])
#     u = batch_normalization(u)
#     z_est[l] = g_gauss(z_c, u, layer_sizes[l])
#     z_est_bn = (z_est[l] - m) / v
#     # append the cost of this layer to d_cost
#     d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z_est_bn - z), 1)) / layer_sizes[l]) * denoising_cost[l])
# print "=== Decoder ==="
